# 如何解析YAML输出
:::info 前提条件
本指南假定您熟悉以下概念：- [聊天模型](/docs/concepts/chat_models)- [输出解析器](/docs/concepts/output_parsers)- [提示模板](/docs/concepts/prompt_templates)- [结构化输出](/docs/how_to/structured_output)- [将可运行对象链式组合](/docs/how_to/sequence/)
:::
来自不同供应商的大语言模型（LLMs）通常因其训练数据的差异而各有所长。这也意味着某些模型在生成非JSON格式的输出时可能表现更"优"、更可靠。
该输出解析器允许用户指定任意模式，并使用YAML格式化响应来查询符合该模式的LLM输出。
:::note请记住，大型语言模型是存在漏洞的抽象概念！你必须使用具备足够能力的LLM才能生成格式正确的YAML。:::

In [ ]:
%pip install -qU langchain langchain-openai

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

我们使用 [Pydantic](https://docs.pydantic.dev) 结合 [`YamlOutputParser`](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.yaml.YamlOutputParser.html#langchain.output_parsers.yaml.YamlOutputParser) 来声明数据模型，并为模型提供更多上下文信息以明确应生成何种类型的 YAML：

In [4]:
from langchain.output_parsers import YamlOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


model = ChatOpenAI(temperature=0)

# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = YamlOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

Joke(setup="Why couldn't the bicycle find its way home?", punchline='Because it lost its bearings!')

解析器将自动解析输出的YAML数据并创建对应的Pydantic模型。我们可以查看解析器的`format_instructions`属性，该属性会被添加到提示词中：

In [5]:
parser.get_format_instructions()

'The output should be formatted as a YAML instance that conforms to the given JSON schema below.\n\n# Examples\n## Schema\n```\n{"title": "Players", "description": "A list of players", "type": "array", "items": {"$ref": "#/definitions/Player"}, "definitions": {"Player": {"title": "Player", "type": "object", "properties": {"name": {"title": "Name", "description": "Player name", "type": "string"}, "avg": {"title": "Avg", "description": "Batting average", "type": "number"}}, "required": ["name", "avg"]}}}\n```\n## Well formatted instance\n```\n- name: John Doe\n  avg: 0.3\n- name: Jane Maxfield\n  avg: 1.4\n```\n\n## Schema\n```\n{"properties": {"habit": { "description": "A common daily habit", "type": "string" }, "sustainable_alternative": { "description": "An environmentally friendly alternative to the habit", "type": "string"}}, "required": ["habit", "sustainable_alternative"]}\n```\n## Well formatted instance\n```\nhabit: Using disposable water bottles for daily hydration.\nsustainabl

你可以且应该在提示的其他部分尝试添加自己的格式提示，以补充或替换默认指令。
## 后续步骤
你现在已经学会了如何提示模型返回 YAML 格式的内容。接下来，可以查看[获取结构化输出的综合指南](/docs/how_to/structured_output)来了解其他相关技术。